In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as torchdata
from sklearn.model_selection import train_test_split

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.set_default_tensor_type('torch.cuda.DoubleTensor')

In [ ]:
os.system('clear')
# Import Data
PROJECT_ROOT = "/content/drive/MyDrive/CG-ODE-main/CG-ODE-main/data/Dec"

features = {'dailyDeaths','Deaths','totCases','cases','1Dose','fullyVac','temperature'}
feature_labels = np.array(list(features))

train = np.load(os.path.join(PROJECT_ROOT,"train.npy"))
test =  np.load(os.path.join(PROJECT_ROOT,"test.npy"))

In [ ]:
#choose the feature you wish to train on
train_one = train[:,:,0]
train_one = train_one.T
days, states = train_one.shape

test_one = test[:,:,0]
test_one = test_one.T
days_test, states = test_one.shape

In [ ]:
#we are only going to take equal windows to predict
window = 7 #how many days we wish to predict into the future

#train
train_x = np.array([])
train_y = np.array([])
for i in range(days-2*window):
  if i==0:
    train_x = train_one[i:i+window,:]
    train_x = np.expand_dims(train_x,axis=0)
    train_y = test_one[i+window:i+2*window,:]
    train_y = np.expand_dims(train_y,axis=0)
  else:
    x = train_one[i:i+window,:]
    x = np.expand_dims(x,axis=0)
    train_x = np.vstack((train_x,x))
    y = train_one[i:i+window,:]
    y = np.expand_dims(y,axis=0)
    train_y = np.vstack((train_y,y))


#test
test_x = np.array([])
test_y = np.array([])
for i in range(days_test-2*window):
  if i==0:
    test_x = test_one[i:i+window,:]
    test_x = np.expand_dims(test_x,axis=0)
    test_y = test_one[i+window:i+2*window,:]
    test_y = np.expand_dims(test_y,axis=0)
  else:
    x = test_one[i:i+window,:]
    x = np.expand_dims(x,axis=0)
    test_x = np.vstack((test_x,x))
    y = test_one[i:i+window,:]
    y = np.expand_dims(y,axis=0)
    test_y = np.vstack((test_y,y))

#make into tensor
train_x = torch.from_numpy(train_y)
train_y = torch.from_numpy(train_y)
test_x = torch.from_numpy(test_y)
test_y = torch.from_numpy(test_y)

#make into double tensor
train_x = torch.DoubleTensor(train_x).to('cuda:0')
train_y = torch.DoubleTensor(train_y).to('cuda:0')
test_x = torch.DoubleTensor(test_x).to('cuda:0')
test_y = torch.DoubleTensor(test_y).to('cuda:0')

In [ ]:
train_x.shape,train_y.shape, test_x.shape,test_y.shape

(torch.Size([219, 7, 50]),
 torch.Size([219, 7, 50]),
 torch.Size([17, 7, 50]),
 torch.Size([17, 7, 50]))

In [ ]:
train_x[0,:,:].shape

torch.Size([7, 50])

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=50, hidden_layer_size=64, output_size=50):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear1 = nn.Linear(hidden_layer_size, 128)
        self.linear2 = nn.Linear(128,output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        #lstm_out, self.hidden_cell = self.lstm(input_seq, self.hidden_cell)
        pred1 = self.linear1(lstm_out.view(len(input_seq), -1))
        pred2 = self.linear2(pred1.view(len(input_seq), -1))
        return pred2

In [ ]:
model = LSTM()
dtype = torch.cuda.DoubleTensor
model.type(dtype).to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model(train_x[0,:,:]).shape

torch.Size([7, 50])

In [ ]:
epochs = 500
model.train()
for i in np.arange(epochs):
  #optimizer.zero_grad()
  #model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
   #                     torch.zeros(1, 1, model.hidden_layer_size))
  for j in range(train_x.shape[0]):
      optimizer.zero_grad()
      model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                      torch.zeros(1, 1, model.hidden_layer_size))

      y_pred = model(train_x[j,:])

      single_loss = loss_function(y_pred, train_y[j,:])
      single_loss.backward()
      optimizer.step()

  if i%25 == 0:
      print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   0 loss: 567288649075.60339355
epoch:  25 loss: 502147921461.06555176
epoch:  50 loss: 388250280536.98059082
epoch:  75 loss: 280924172059.73437500
epoch: 100 loss: 212629267737.92102051
epoch: 125 loss: 183728429896.71392822
epoch: 150 loss: 176344409730.77276611
epoch: 175 loss: 176268355259.80737305
epoch: 200 loss: 177562315367.40628052
epoch: 225 loss: 178612583928.94409180
epoch: 250 loss: 179215139215.39749146
epoch: 275 loss: 179479631401.11566162
epoch: 300 loss: 179528803602.72222900
epoch: 325 loss: 179446298201.49673462
epoch: 350 loss: 179282399223.92166138
epoch: 375 loss: 179065734236.38351440
epoch: 400 loss: 178812234943.31292725
epoch: 425 loss: 178530765927.06713867
epoch: 450 loss: 178226356901.02410889
epoch: 475 loss: 177901981034.36068726
epoch: 499 loss: 177573541052.1642761230


In [ ]:
model.eval()
mape_list = []
for i in range(test_x.shape[0]):
  y_pred_test = model(test_x[i,:])
  mape_vec = torch.abs(y_pred_test - test_y[i,:])/test_y[i,:]
  mape_np = mape_vec.detach().cpu().numpy()
  mape_list.append(np.mean(mape_np))

np.mean(mape_list)

2.285555262128937

In [ ]:
y_pred_test = model(test_x[0,:])
mape_vec = torch.abs(y_pred_test - test_y[0,:])/test_y[0,:]
mape_np = mape_vec.detach().cpu().numpy()
np.mean(mape_np)

2.3387388433665426